In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import datetime
import statistics as stats
import tsfresh.feature_extraction.feature_calculators as fc
from functools import reduce
import math

### Daten einlesen und Spalten entfernen welche nicht benötigt werden.

In [2]:
reib_daten = pd.read_csv('data_2019-26-11_TRAININGDATA.csv')
achsleistung_daten = pd.read_csv('../Achsleistungen _ Messprotokolle/Ebene 1/Achsleistung-2019-11-20T10-27-03.csv', \
                                 delimiter=';')
messprotokoll_daten = pd.read_csv('../Achsleistungen _ Messprotokolle/Ebene 1/MEAS_PROTOCOL_CSV.CSV', delimiter=';')
messergebnisse_daten = pd.read_csv('../Messergebnisse Reiben/Messergebnisse.csv', skiprows=7, delimiter=';')
achsleistung_columns = ['Timestamp', 'X1', 'Y1', 'Z1', 'SPI1']
messergebnisse_columns = ['Messwert Innenmessschraube', 'Messwert Messtaster', 'Klassifizierung nach Messtaster', \
                          'Klassifizierung nach Messtaster', 'Klassifizierung nach Grenzlehrdorn']
achsleistung_daten = achsleistung_daten[achsleistung_columns]
messergebnisse_daten = messergebnisse_daten[messergebnisse_columns]

### Hilfestellung zur Ermittlung von Löchern
Die Funktion gibt die Intervalle für die gewünschten Bohrungen zurück. Zu beachten:
- keine negativen Zahlen
- Es muss für beide Funktionen dieselbe Ebene gewählt werden, da die Funktion ```bohrungenDavor()```mit 81 Bohrungen weniger rechnet!

In [3]:
def bohrungEbene(ebene):
    aktuelle_bohrung = 81 * ebene
    return aktuelle_bohrung

def bohrungenDavor(ebene):
    if ebene == 0 or ebene == 1:
        return 0
    elif ebene > 1:
        bohrungen_davor = (ebene - 1) * 81
        return bohrungen_davor

    
print("Anz. Bohrungen davor: " + str(bohrungenDavor(2)) + "\nAnz. Bohrungen aktuell: " + str(bohrungEbene(2)))

Anz. Bohrungen davor: 81
Anz. Bohrungen aktuell: 162


### Anpassen und aufbereiten Messprotokolldaten
Da die Maschine die Werte in der Exceltabelle sehr schlecht formatiert - viele Tabulatoren und alle Zahlen sind strings. Zusätzlich werden noch alle null values entfernt.

In [4]:
messprotokoll_daten = messprotokoll_daten.rename(columns=lambda x: x.strip())
messprotokoll_columns = ['Measured', 'Difference']
messprotokoll_daten = messprotokoll_daten[messprotokoll_columns]

for column, value in enumerate(messprotokoll_columns):
    messprotokoll_daten[value] = messprotokoll_daten[value].str.strip()
    messprotokoll_daten[value] = messprotokoll_daten[value].str.replace(',', '.')
    messprotokoll_daten[value] = messprotokoll_daten[value].apply(pd.to_numeric, errors='coerce')

print(messprotokoll_daten.dtypes)

achsleistung_daten = achsleistung_daten.dropna()
messprotokoll_daten = messprotokoll_daten.dropna()
messergebnisse_daten = messergebnisse_daten.dropna()

Measured      float64
Difference    float64
dtype: object


### Berechnung Mittelwert und Median für Messprotokolldaten

Hier werden die Daten für das Messprotokoll angepasst. Da hier eine Bohrung aus drei Werten besteht (Spalte Result: X, Y, Diameter), werden alle drei Werte genommen und aggregiert $\rightarrow$ Median + Mittelwert.

Danach wird das Messprotokoll mit den Daten der Messergebnisse gemerged.

In [5]:
filling_columns = ['Measured Mean', 'Measured Median', 'Difference Mean', 'Difference Median']

messprotokoll_condensed = pd.DataFrame(columns=filling_columns)
measured_avg = []
measured_median = []
diameter_avg = []
diameter_median = []
count = 1
for key, value in messprotokoll_daten.iteritems():
    if key == 'Measured':
        meas = []
        for index, floating in enumerate(value):
            meas.append(floating)
            if (count % 3) == 0:
                mean_meas = stats.mean(meas)
                median_meas = stats.median(meas)
                measured_avg.append(mean_meas)
                measured_median.append(median_meas)
                #print(measured_avg, '\n', measured_median)
                del meas[:]
                
            #print(count % 3 == 0, meas, count)
            if count == 1:
                pass
            
            count += 1

    elif key == 'Difference':
        diff = []
        for index, floating in enumerate(value):
            diff.append(abs(floating))
            if (count % 3) == 0:
                mean_diff = np.mean(diff)
                median_diff = np.median(diff)
                diameter_avg.append(mean_diff)
                diameter_median.append(median_diff)
                del diff[:]
            
            if count == 1:
                pass
            
            count += 1

            

messprotokoll_condensed['Measured Mean'] = measured_avg
messprotokoll_condensed['Measured Median'] = measured_median
messprotokoll_condensed['Difference Mean'] = diameter_avg
messprotokoll_condensed['Difference Median'] = diameter_median

#AUFPASSEN, HIER MUSS DIE RICHTIGE EBENE ADRESSIERT WERDEN!

messergebnisse_ebene = messergebnisse_daten[bohrungenDavor(1):bohrungEbene(1)]

merge_csv_erg_prot = messergebnisse_ebene.reset_index().merge(messprotokoll_condensed, left_index=True, right_index=True)
merge_csv_erg_prot

,index,Messwert Innenmessschraube,Messwert Messtaster,Klassifizierung nach Messtaster,Klassifizierung nach Messtaster,Klassifizierung nach Grenzlehrdorn,Measured Mean,Measured Median,Difference Mean,Difference Median
0,0,"12,016","12,019",n.i.O.,n.i.O.,i.O.,14.003667,14.976,0.016667,0.016
1,1,"12,016","12,018",i.O.,i.O.,i.O.,18.999333,14.986,0.009667,0.009
2,2,"12,016","12,015",i.O.,i.O.,i.O.,23.997000,14.997,0.010000,0.006
3,3,"12,016","12,015",i.O.,i.O.,i.O.,29.005000,15.003,0.004000,0.003
4,4,"12,016","12,015",i.O.,i.O.,i.O.,33.996000,15.004,0.013667,0.006
...,...,...,...,...,...,...,...,...,...,...
76,76,"12,015","12,018",i.O.,i.O.,i.O.,74.003333,74.988,0.008333,0.009
77,77,"12,015","12,017",i.O.,i.O.,i.O.,78.998667,89.982,0.009667,0.008
78,78,"12,015","12,018",i.O.,i.O.,i.O.,84.008333,105.001,0.005333,0.006
79,79,"12,016","12,018",i.O.,i.O.,i.O.,89.008000,120.005,0.004667,0.005


### Achsleitungsverarbeitung und entfernung Ausreiser?
Hier wird die Achleistung bearbeitet. Diskutierbar ist, ob "Ausreiser" die durch Median * 3 ermittelt werden, entfernt werden müssen oder nicht. Es macht keine Unterschiede in der Anzahl der Daten, jedoch in deren Aggregierten Werten (Max, Mean, Median).

In [6]:
# delete any row with 0's in SPI
achsleistung_daten = achsleistung_daten[(achsleistung_daten[['SPI1']] != 0).all(axis=1)]

# remove median > 3 for all columns
for key, value in achsleistung_daten.items():
    if achsleistung_daten[key].dtypes == 'float64':
        achsleistung_daten.drop(achsleistung_daten[value > (3 * stats.mean(achsleistung_daten[key]))].index, inplace=True)
        

achsleistung_daten = achsleistung_daten.set_index(['Timestamp'])

In [ ]:
# achsleistung_daten.index = pd.to_datetime(achsleistung_daten.index)
# print(achsleistung_daten.index[-1], achsleistung_daten.index[0])
# time_diff = (achsleistung_daten.index[-1] - achsleistung_daten.index[0])

# resampling_range = time_diff.total_seconds() / 81

# print(math.floor(resampling_range))

# index_series = pd.date_range(start=achsleistung_daten.index[0], end=achsleistung_daten.index[-1], periods=time_diff.total_seconds())

# achsleistung_daten = achsleistung_daten.reindex(achsleistung_daten.index.union(index_series)).interpolate(method='time').reindex(index_series)

# achsleistung_daten
#dateRangeAndAggregate()

### Aufteilung der Daten in gleich große Zeitintervalle und Aggregierung der Daten
Hier wird zunächst die Gesamtzeit zwischen Start und Ende der Achsleistung bestimmt. Diese wird durch 81 geteilt (um einen Zeitraum für eine Bohrung bestimmen zu können), und mit diesem Zeitraum werden die Daten auf verschiedenen Aggregationen - Max, Min, Mean, Median - neu gesampled.
Die berechneten Werte werden dann dem zuvor erstellten DataFrame "merge_csv_erg_prot" hinzugefügt.

Die Datenverarbeitung ist damit abgeschlossen.

In [7]:
def reduce_df(dataframe):
    reduced_frame = reduce(lambda left,right: pd.merge(left,right,on='Timestamp'), dataframe)
    return reduced_frame

achsleistung_daten.index = pd.to_datetime(achsleistung_daten.index)

In [8]:
time_diff = (achsleistung_daten.index[-1] - achsleistung_daten.index[0])
resampling_range = (time_diff.total_seconds() / 81)

print('resampling range: '+ str(resampling_range) + '\ntime difference: ' + str(time_diff) + \
      '\ntime of one drill: ' + str((time_diff / resampling_range)))

sampled_mean = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').mean()
sampled_median = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').median()
sampled_max = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').max()
sampled_min = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').min()
#sampled_app_entropy = achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').apply(fc.approximate_entropy(achsleistung_daten), 81, r=1.0)
#sampled_std_deviation = fc.standard_deviation(achsleistung_daten.resample(str(math.ceil(resampling_range))+'S').std())

sampled_mean = sampled_mean.rename(
    columns={'X1': 'X1_mean', 'Y1': 'Y1_mean', 'Z1': 'Z1_mean', 'SPI1': 'SPI1_mean'})
sampled_median = sampled_median.rename(
    columns={'X1': 'X1_median', 'Y1': 'Y1_median', 'Z1': 'Z1_median', 'SPI1': 'SPI1_median'})
sampled_max = sampled_max.rename(                         
    columns={'X1': 'X1_max', 'Y1': 'Y1_max', 'Z1': 'Z1_max', 'SPI1': 'SPI1_max'})
sampled_min = sampled_min.rename(                         
    columns={'X1': 'X1_min', 'Y1': 'Y1_min', 'Z1': 'Z1_min', 'SPI1': 'SPI1_min'})
        
        
df_list = [sampled_mean, sampled_median, sampled_max, sampled_min]
df_final = reduce_df(df_list)

df_final = df_final.reset_index()

df_final

resampling range: 27.965185185185184
time difference: 0 days 00:37:45.180000
time of one drill: 0 days 00:01:21


,Timestamp,X1_mean,Y1_mean,Z1_mean,SPI1_mean,X1_median,Y1_median,Z1_median,SPI1_median,X1_max,Y1_max,Z1_max,SPI1_max,X1_min,Y1_min,Z1_min,SPI1_min
0,2019-11-20 10:26:44,392.248571,441.383643,3319.495357,992.799500,380.0700,538.6120,3148.3840,936.7275,625.162,718.150,5556.411,1398.495,62.161,112.651,3051.165,705.844
1,2019-11-20 10:27:12,479.724806,542.262861,3345.313167,1007.642083,521.2640,664.1710,3071.1070,1019.1865,710.411,746.312,7253.995,1411.688,136.754,2.347,894.909,659.667
2,2019-11-20 10:27:40,499.276920,547.765040,2835.190360,995.570160,563.0010,638.3550,3073.6000,996.0980,721.067,751.006,3594.591,1398.495,103.010,39.897,254.264,646.474
3,2019-11-20 10:28:08,464.431265,558.629794,3367.528353,951.861412,530.1445,684.1195,3112.2380,933.4295,772.572,748.659,6197.056,1385.302,10.656,30.510,3001.309,593.701
4,2019-11-20 10:28:36,454.192971,501.476000,3262.904824,950.503147,526.5920,582.0295,3074.8465,946.6225,706.859,751.006,8228.673,1352.318,24.864,16.428,147.074,633.281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2019-11-20 11:02:40,117.054096,109.911167,3430.120004,382.662314,131.4260,63.3660,3906.1890,422.1870,209.571,743.965,6882.571,864.164,15.984,2.347,974.678,6.597
78,2019-11-20 11:03:08,133.513698,145.718118,3476.626898,383.576433,134.9780,124.3850,3916.1600,415.5910,197.139,657.130,5763.312,949.921,81.697,0.000,1652.714,26.387
79,2019-11-20 11:03:36,138.771719,119.062430,3373.603698,391.702089,147.4100,100.9160,2747.0460,428.7840,170.499,323.871,9709.385,804.794,21.312,0.000,2540.145,13.193
80,2019-11-20 11:04:04,97.429803,115.718476,3261.118063,373.932740,95.9060,119.6920,2747.0460,389.2040,186.483,380.197,7817.364,1035.678,60.385,0.000,2659.798,13.193


In [9]:
merge_csv_erg_prot = merge_csv_erg_prot.merge(df_final, right_index=True, left_index=True)

In [10]:
merge_csv_erg_prot.reset_index(drop=True)

,index,Messwert Innenmessschraube,Messwert Messtaster,Klassifizierung nach Messtaster,Klassifizierung nach Messtaster,Klassifizierung nach Grenzlehrdorn,Measured Mean,Measured Median,Difference Mean,Difference Median,...,Z1_median,SPI1_median,X1_max,Y1_max,Z1_max,SPI1_max,X1_min,Y1_min,Z1_min,SPI1_min
0,0,"12,016","12,019",n.i.O.,n.i.O.,i.O.,14.003667,14.976,0.016667,0.016,...,3148.3840,936.7275,625.162,718.150,5556.411,1398.495,62.161,112.651,3051.165,705.844
1,1,"12,016","12,018",i.O.,i.O.,i.O.,18.999333,14.986,0.009667,0.009,...,3071.1070,1019.1865,710.411,746.312,7253.995,1411.688,136.754,2.347,894.909,659.667
2,2,"12,016","12,015",i.O.,i.O.,i.O.,23.997000,14.997,0.010000,0.006,...,3073.6000,996.0980,721.067,751.006,3594.591,1398.495,103.010,39.897,254.264,646.474
3,3,"12,016","12,015",i.O.,i.O.,i.O.,29.005000,15.003,0.004000,0.003,...,3112.2380,933.4295,772.572,748.659,6197.056,1385.302,10.656,30.510,3001.309,593.701
4,4,"12,016","12,015",i.O.,i.O.,i.O.,33.996000,15.004,0.013667,0.006,...,3074.8465,946.6225,706.859,751.006,8228.673,1352.318,24.864,16.428,147.074,633.281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,76,"12,015","12,018",i.O.,i.O.,i.O.,74.003333,74.988,0.008333,0.009,...,2739.5670,395.8000,136.754,307.443,10060.868,903.744,40.849,0.000,2657.305,6.597
77,77,"12,015","12,017",i.O.,i.O.,i.O.,78.998667,89.982,0.009667,0.008,...,3906.1890,422.1870,209.571,743.965,6882.571,864.164,15.984,2.347,974.678,6.597
78,78,"12,015","12,018",i.O.,i.O.,i.O.,84.008333,105.001,0.005333,0.006,...,3916.1600,415.5910,197.139,657.130,5763.312,949.921,81.697,0.000,1652.714,26.387
79,79,"12,016","12,018",i.O.,i.O.,i.O.,89.008000,120.005,0.004667,0.005,...,2747.0460,428.7840,170.499,323.871,9709.385,804.794,21.312,0.000,2540.145,13.193


In [11]:
# write to csv
merge_csv_erg_prot.to_csv('./Messergebnisse_merged.csv')
merge_csv_erg_prot.to_json('./Messergebnisse_merged.json', orient='split')